# Compare EC profit redistribution

This notebook aims at comparing the profit redistribution for an Energy Community
using the package EnergyCommunity.jl and Games.jl

## Configuration

### Paths

In [ ]:
input_file = joinpath(@__DIR__, "../data/energy_community_model.yml")  # Input file

output_file_isolated = joinpath(@__DIR__, "../results/output_file_NC.xlsx")  # Output file - model users alone
output_plot_isolated = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_NC.png")  # Output png file of plot - model users alone

output_file_combined = joinpath(@__DIR__, "../results/output_file_EC.xlsx")  # Output file - model Energy community
output_plot_combined = joinpath(@__DIR__, "../results/Img/plot_user_{:s}_EC.pdf")  # Output png file of plot - model energy community

output_plot_sankey_agg = joinpath(@__DIR__, "../results/Img/sankey_EC.png")  # Output plot of the sankey plot related to the aggregator case
output_plot_sankey_noagg = joinpath(@__DIR__, "../results/Img/sankey_NC.png")  # Output plot of the sankey plot related to the no aggregator case

enum_mode_file = "enum_mode_datasest.jld2"  # file used to store the enumerative results
total_results_file = "total_results_file_500.jld2"  # file to store all major results
latex_output = "latex_output3.txt"

overwrite_files = true  # when true, output files are overwritten

### Import libraries

In [ ]:
using Revise
using EnergyCommunity
using FileIO
using HiGHS, Plots
using JuMP
using Gurobi
using Games
using TickTock
using Combinatorics
using DataFrames
using JLD2
using Latexify, LaTeXStrings

### Solver configurations

In [ ]:
# General optimizer
OPTIMIZER = optimizer_with_attributes(Gurobi.Optimizer, "OutputFlag"=>0, "Threads"=>10)

# Optimizer for row-generation techniques, used in the IterMode of Games.jl
OPTIMIZER_ROW_GENERATION = optimizer_with_attributes(Gurobi.Optimizer,
    "OutputFlag"=>1,
    "LogToConsole"=>0,
    "MIPGap"=>0.1,
    # "MIPFocus"=>1,
    "TimeLimit"=>1000,
    "LogFile"=>"gurobi.log",
    "Threads"=>10,
    # "NoRelHeurTime"=>10,
    "PoolSolutions"=>500,
    "PoolSearchMode"=>1,
)

### Energy Community options

In [ ]:
NO_AGG_GROUP = GroupANC();  # type of aggregation when the Aggregator does not belong to the coalition.
                            # options: GroupANC() or GroupNC()
BASE_GROUP = GroupNC();     # base type of aggregation (it shall be GroupNC)

## General

In [ ]:
ENV["COLUMNS"] = 1000  # to print more columns in the output

## Optimize the Energy Community in the COperative configuration

In [ ]:
# Read data from excel file
ECModel = ModelEC(input_file, EnergyCommunity.GroupCO(), OPTIMIZER)

# Reset the user set to use all stored users (10)
reset_user_set!(ECModel)
# set_user_set!(ECModel, ["user$id" for id=1:8])

# Build the model
build_model!(ECModel)

# Optimize the model
optimize!(ECModel)

## Calculate the reward distribution functions by enumerative techniques (EnumMode in Games.jl)

### Create enumerative mode

Calculation of the enumerative mode used to compute several reward distribution functions

In [ ]:
# if !isfile(total_results_file) || overwrite_files
#     tick()
#     enum_mode = EnumMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP)
#     time_elapsed_enum=tok()
#     println("EnumMode calculated with elapsed time [min]: $(time_elapsed_enum/60)")
# else
time_elapsed_enum=0.0
# end

Save enum mode

In [ ]:
# if !isfile(total_results_file) || overwrite_files
#     save(enum_mode_file, enum_mode)
# end

Load EnumMode (when needed)

In [ ]:
# enum_mode = load(enum_mode_file, EnumMode())

### Calculate reward distribution using EnumMode

In [ ]:
tick()
shapley_dist_enum = shapley_value(enum_mode)  # shapley value
time_elapsed_shapley_enum=tok()

tick()
nucleolus_dist_enum, n_iterations_nucleolus_enum, model_nucleolus_enum = nucleolus(enum_mode, OPTIMIZER; raw_outputs=true)  # nucleolus
time_elapsed_nucleolus_enum=tok()

tick()
varcore_dist_enum = var_in_core(enum_mode, OPTIMIZER)  # variance in core
time_elapsed_varcore_enum=tok()

tick()
varleastcore_dist_enum, val_minsurplus_enum, model_dist_enum = var_least_core(
    enum_mode, OPTIMIZER; raw_outputs=true
)  # variance least core (include raw outputs for comparison purposes)
time_elapsed_varleastcore_enum=tok();

Store the EnumMode reward distribution into a DataFrame for ease their use

In [ ]:
# vector of the users
user_set_agg = [EC_CODE; get_user_set(ECModel)]

"Auxiliary function to order the output of reward distributions and return them as vectors"
vectorize_rewards(reward_dist, users_list=user_set_agg) = [reward_dist[u] for u in users_list]

# dataframe of reward distributions for the enumerative mode
df_reward_enum = DataFrame(
    user_set=user_set_agg,
    shapley_enum=vectorize_rewards(shapley_dist_enum),
    nucleolus_enum=vectorize_rewards(nucleolus_dist_enum),
    varcore_enum=vectorize_rewards(varcore_dist_enum),
    varleastcore_enum=vectorize_rewards(varleastcore_dist_enum),
)

# dataframe of the time requirements
dict_time_enum = Dict(
    "EnumMode"=>time_elapsed_enum,
    "shapley_enum"=>time_elapsed_shapley_enum+time_elapsed_enum,
    "nucleolus_enum"=>time_elapsed_nucleolus_enum+time_elapsed_enum,
    "varcore_enum"=>time_elapsed_varcore_enum+time_elapsed_enum,
    "varleastcore_enum"=>time_elapsed_varleastcore_enum+time_elapsed_enum,
)

df_reward_enum

## Calculate the reward distribution functions by row-generation techniques (IterMode in Games.jl)

### Define the row-generation decomposition mode by IterMode

In [ ]:
iter_mode = IterMode(ECModel, BASE_GROUP; no_aggregator_group=NO_AGG_GROUP, optimizer=OPTIMIZER_ROW_GENERATION)

Define coalitions to precompute when performing the iterative procedure

In [ ]:
# include all coalitions having no more than preload_max_size users
preload_max_size = 2

preload_coalitions = Iterators.flatten([combinations([EC_CODE; ECModel.user_set], k) for k = 1:preload_max_size])

### Calculate reward distribution using row-generation technique

Variance Least Core using IterMode

In [ ]:
tick()
varleastcore_dist_iter, min_surplus_varleastcore_iter, history_varleastcore_iter, model_dist_varleastcore_iter = var_least_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
)
time_elapsed_varleastcore_iter=tok()
println("Variance Least Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varleastcore_iter/60)")

Variance Core method using IterMode

In [ ]:
tick()
varcore_dist_iter, min_surplus_varcore_iter, history_varcore_iter, model_dist_varcore_iter = var_in_core(
    iter_mode,
    OPTIMIZER;
    lower_bound=0.0,
    atol=1e-4,
    raw_outputs=true,
    preload_coalitions=preload_coalitions,
)
time_elapsed_varcore_iter=tok()
println("Variance Core - IterMode calculated with elapsed time [min]: $(time_elapsed_varcore_iter/60)")

Store results as a DataFrame

In [ ]:
# dataframe of reward distributions for the enumerative mode
df_reward_iter = DataFrame(
    user_set=user_set_agg,
    varcore_iter=vectorize_rewards(varcore_dist_iter),
    varleastcore_iter=vectorize_rewards(varleastcore_dist_iter),
)

# dataframe of the time requirements
dict_time_iter = Dict(
    "IterMode"=>0.0,
    "varcore_iter"=>time_elapsed_varcore_iter,
    "varleastcore_iter"=>time_elapsed_varleastcore_iter,
)

df_reward_iter

## Group all results and save them

Merge results

In [ ]:
df_reward = innerjoin(df_reward_enum, df_reward_iter, on=:user_set)
df_reward

In [ ]:
dict_time = merge(dict_time_enum, dict_time_iter)
dict_time

Save results

In [ ]:
# print to latex the reward table
set_default(fmt = "%.2f", convert_unicode = false)

sorted_user_set = [EC_CODE; ["user$u" for u in 1:length(get_user_set(ECModel))]]

df_reward_mod = copy(df_reward)
sort!(df_reward_mod, [order(:user_set, by=x->findfirst(x .== sorted_user_set))])
df_reward_mod[!, 2:end] = df_reward[!, 2:end] ./ 1000  # change € unit to k€
tex_df_reward = latexify(df_reward_mod; env=:table, latex=false)

In [ ]:
df_time = DataFrame(dict_time)[!, names(df_reward)[2:end]]./3600
df_time[!, "title"] = [L"Time [h]"]
df_time = df_time[!, ["title"; names(df_reward)[2:end]]]

In [ ]:
df_iterations = DataFrame(
    title=L"Iterations",
    shapley_enum=L"-",
    nucleolus_enum=n_iterations_nucleolus_enum,
    varcore_enum=L"-",
    varleastcore_enum=L"-",
    varcore_iter=history_varcore_iter[end][1],
    varleastcore_iter=history_varleastcore_iter[end][1],
)

In [ ]:
df_computational_time = vcat(df_time, df_iterations)

# print to latex the equivalent dataframe for time
tex_df_computational_time = latexify(df_computational_time; env=:table)

In [ ]:
# save latex code
open(latex_output,"w") do io
    println(io, "Reward distribution table\n\n\n")
    print(io, tex_df_reward)
    println(io, "Computational time table\n\n\n")
    print(io, tex_df_computational_time)
end

In [ ]:
if !isfile(total_results_file) || overwrite_files
    jldsave(total_results_file; df_reward, dict_time, df_computational_time)
end